# tf.data を使って pandas の DataFrame をロードする

https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

In [23]:
import pandas as pd
import tensorflow as tf

In [24]:
#csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')
csv_file = tf.keras.utils.get_file('heart.csv', 'https://raw.githubusercontent.com/rashida048/Datasets/master/Heart.csv')

In [25]:
df = pd.read_csv(csv_file, index_col=0).reset_index(drop=True)
df.columns = df.columns.str.lower()

In [26]:
df.head()

,age,sex,chestpain,restbp,chol,fbs,restecg,maxhr,exang,oldpeak,slope,ca,thal,ahd
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


In [27]:
df.dtypes

age            int64
sex            int64
chestpain     object
restbp         int64
chol           int64
fbs            int64
restecg        int64
maxhr          int64
exang          int64
oldpeak      float64
slope          int64
ca           float64
thal          object
ahd           object
dtype: object

In [28]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes
df['chestpain'] = pd.Categorical(df['chestpain'])
df['chestpain'] = df.chestpain.cat.codes
df['ahd'] = pd.Categorical(df['ahd'])
df['ahd'] = df.ahd.cat.codes

In [29]:
df.head()

,age,sex,chestpain,restbp,chol,fbs,restecg,maxhr,exang,oldpeak,slope,ca,thal,ahd
0,63,1,3,145,233,1,2,150,0,2.3,3,0.0,0,0
1,67,1,0,160,286,0,2,108,1,1.5,2,3.0,1,1
2,67,1,0,120,229,0,2,129,1,2.6,2,2.0,2,1
3,37,1,1,130,250,0,0,187,0,3.5,3,0.0,1,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,1,0


In [30]:
target = df.pop('ahd')

In [31]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [33]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [ 63.    1.    3.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   0. ], Target: 0
Features: [ 67.    1.    0.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   1. ], Target: 1
Features: [ 67.    1.    0.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   2. ], Target: 1
Features: [ 37.    1.    1.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   1. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   1. ], Target: 0


In [34]:
tf.constant(df['thal'])

<tf.Tensor: shape=(303,), dtype=int8, numpy=
array([ 0,  1,  2,  1,  1,  1,  1,  1,  2,  2,  0,  1,  0,  2,  2,  1,  2,
        1,  1,  1,  1,  1,  1,  2,  2,  1,  1,  1,  1,  2,  1,  2,  1,  2,
        1,  1,  2,  0,  2,  1,  2,  2,  1,  1,  1,  2,  1,  2,  1,  1,  1,
        2,  1,  1,  2,  2,  2,  2,  1,  1,  2,  1,  2,  1,  2,  2,  1,  2,
        2,  1,  1,  2,  2,  0,  1,  1,  2,  1,  1,  2,  1,  1,  1,  2,  1,
        1,  1, -1,  1,  1,  1,  2,  2,  1,  1,  2,  2,  2,  1,  1,  1,  1,
        1,  1,  2,  2,  2,  2,  2,  2,  2,  1,  0,  2,  2,  0,  1,  1,  2,
        2,  2,  2,  1,  2,  1,  1,  2,  2,  1,  1,  2,  2,  1,  1,  1,  1,
        2,  2,  2,  1,  1,  2,  1,  2,  2,  1,  2,  1,  1,  1,  2,  1,  2,
        2,  1,  1,  2,  2,  2,  2,  2,  1,  1,  1,  1,  2,  1,  1,  2,  1,
        2,  2,  1,  1,  0,  2,  2,  0,  1,  1,  2,  2,  1,  2,  1,  1,  2,
        0,  2,  2,  1,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  1,  1,  1,  2,  1,  2,  1,  2, 

In [35]:
train_dataset = dataset.shuffle(len(df)).batch(1)

In [36]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [37]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Epoch 1/15
303/303 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.5314
Epoch 2/15
303/303 [==============================] - 0s 993us/step - loss: nan - accuracy: 0.5413
Epoch 3/15
303/303 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5413
Epoch 4/15
303/303 [==============================] - 0s 949us/step - loss: nan - accuracy: 0.5413
Epoch 5/15
303/303 [==============================] - 0s 938us/step - loss: nan - accuracy: 0.5413
Epoch 6/15
303/303 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5413
Epoch 7/15
303/303 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5413
Epoch 8/15
303/303 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5413
Epoch 9/15
303/303 [==============================] - 0s 1ms/step - loss: nan - accuracy: 0.5413
Epoch 10/15
303/303 [==============================] - 0s 934us/step - loss: nan - accuracy: 0.5413
Epoch 11/15
303/303 [

In [38]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [39]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [40]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57],
      dtype=int32)>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1], dtype=int32)>, 'chestpain': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([3, 0, 0, 1, 2, 2, 0, 0, 0, 0, 0, 2, 1, 2, 1, 1], dtype=int32)>, 'restbp': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150], dtype=int32)>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168], dtype=int32)>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int32)>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0], dtype=int32)>, 'maxhr': <tf.Tensor: shape=

In [41]:
model_func.fit(dict_slices, epochs=15)

Epoch 1/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5215
Epoch 2/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 3/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 4/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 5/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 6/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 7/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 8/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 9/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 10/15
19/19 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5413
Epoch 11/15
19/19 [==============================